In [1]:
root_dir = "F:/data/"

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import backend
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation, Concatenate, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers, activations

import os

from sklearn.utils import shuffle
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

import sys

import matplotlib.image as mpimg
from skimage.transform import resize

from keras import models
from keras.layers.core import Activation, Reshape, Permute
from keras.layers.convolutional import Convolution2D, MaxPooling2D, UpSampling2D
from keras.layers.normalization import BatchNormalization
import json


Using TensorFlow backend.


In [3]:
train_batch_size = 8
val_batch_size = 8

IMAGE_SIZE_WIDTH = 1600
IMAGE_SIZE_HEIGHT = 256

smooth = 1.

train_image_dir = root_dir + "train_images/"
test_image_dirs = [root_dir + str(i)+'/' for i in range(1,5)]
print(train_image_dir)
print(test_image_dirs)

F:/data/train_images/
['F:/data/1/', 'F:/data/2/', 'F:/data/3/', 'F:/data/4/']


In [4]:
def load_image(infilename):
  """ Load an image from disk. """
  return mpimg.imread(infilename)


In [5]:
files = os.listdir(image_dir)
n = 10#len(files)

print("Loading " + str(n) + " images from",image_dir)

tt=[]

fail=0
for i in range(n):
    try:
        tt.append(load_image(train_image_dir + files[i]))
        print(files[i])
        sys.stdout.write("\033[F")
    except Exception as e:
        print("Failed at",train_image_dir,files[i])
        print(e)
print(len(tt),"Train images Loaded")

X = np.asarray(tt)
print(X.shape)


NameError: name 'image_dir' is not defined

In [ ]:
def load_train_images(gt_dir,n):
    img_data_list = [] 
    labels_list = []

    import cv2

    img_data_list = [] 
    labels_list = []
    for i in range(n):
        input_img=cv2.imread(gt_dir + files[i])
        input_img=cv2.cvtColor(input_img, cv2.COLOR_BGR2GRAY)
        #mask = img_to_array(load_img(path_train + '//masks//' + id_, grayscale=False))
        input_img = resize(input_img, (256,1600,3), mode='constant', preserve_range=True)
        #input_img_resize=cv2.resize(input_img,(128,128))
        img_data_list.append(input_img)
        #labels_list.append(label)
    
    y = np.array(img_data_list)
    y = y.astype('float32')
    y /= 255
    
    return y

In [ ]:
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

In [ ]:
def precision(y_true, y_pred):
    """Precision metric.
    Only computes a batch-wise average of precision.
    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def recall(y_true, y_pred):
    """Recall metric.
    Only computes a batch-wise average of recall.
    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def f1score(y_true, y_pred):
    precision_n = precision(y_true, y_pred)
    recall_n = recall(y_true, y_pred)
    return 2 * ((precision_n * recall_n) / (precision_n + recall_n))

In [ ]:
from keras import models
from keras.layers.core import Activation, Reshape, Permute
from keras.layers.convolutional import Convolution2D, MaxPooling2D, UpSampling2D
from keras.layers.normalization import BatchNormalization
import json

img_w = IMAGE_SIZE_WIDTH
img_h = IMAGE_SIZE_HEIGHT
n_labels = 4


def build_model():

    kernel = 3

    encoding_layers = [
        Convolution2D(64, kernel, border_mode='same', input_shape=( img_h, img_w,3)),
        BatchNormalization(),
        Activation('relu'),
        Convolution2D(64, kernel, border_mode='same'),
        BatchNormalization(),
        Activation('relu'),
        MaxPooling2D(),

        Convolution2D(128, kernel, kernel, border_mode='same'),
        BatchNormalization(),
        Activation('relu'),
        Convolution2D(128, kernel, kernel, border_mode='same'),
        BatchNormalization(),
        Activation('relu'),
        MaxPooling2D(),

        Convolution2D(256, kernel, kernel, border_mode='same'),
        BatchNormalization(),
        Activation('relu'),
        Convolution2D(256, kernel, kernel, border_mode='same'),
        BatchNormalization(),
        Activation('relu'),
        Convolution2D(256, kernel, kernel, border_mode='same'),
        BatchNormalization(),
        Activation('relu'),
        MaxPooling2D(),

        Convolution2D(512, kernel, kernel, border_mode='same'),
        BatchNormalization(),
        Activation('relu'),
        Convolution2D(512, kernel, kernel, border_mode='same'),
        BatchNormalization(),
        Activation('relu'),
        Convolution2D(512, kernel, kernel, border_mode='same'),
        BatchNormalization(),
        Activation('relu'),
        MaxPooling2D(),

        Convolution2D(512, kernel, kernel, border_mode='same'),
        BatchNormalization(),
        Activation('relu'),
        Convolution2D(512, kernel, kernel, border_mode='same'),
        BatchNormalization(),
        Activation('relu'),
        Convolution2D(512, kernel, kernel, border_mode='same'),
        BatchNormalization(),
        Activation('relu'),
        MaxPooling2D(),
    ]

    autoencoder = models.Sequential()
    autoencoder.encoding_layers = encoding_layers

    for l in autoencoder.encoding_layers:
        autoencoder.add(l)
        print(l.input_shape,l.output_shape,l)

    decoding_layers = [
        UpSampling2D(),
        Convolution2D(512, kernel, kernel, border_mode='same'),
        BatchNormalization(),
        Activation('relu'),
        Convolution2D(512, kernel, kernel, border_mode='same'),
        BatchNormalization(),
        Activation('relu'),
        Convolution2D(512, kernel, kernel, border_mode='same'),
        BatchNormalization(),
        Activation('relu'),

        UpSampling2D(),
        Convolution2D(512, kernel, kernel, border_mode='same'),
        BatchNormalization(),
        Activation('relu'),
        Convolution2D(512, kernel, kernel, border_mode='same'),
        BatchNormalization(),
        Activation('relu'),
        Convolution2D(256, kernel, kernel, border_mode='same'),
        BatchNormalization(),
        Activation('relu'),

        UpSampling2D(),
        Convolution2D(256, kernel, kernel, border_mode='same'),
        BatchNormalization(),
        Activation('relu'),
        Convolution2D(256, kernel, kernel, border_mode='same'),
        BatchNormalization(),
        Activation('relu'),
        Convolution2D(128, kernel, kernel, border_mode='same'),
        BatchNormalization(),
        Activation('relu'),

        UpSampling2D(),
        Convolution2D(128, kernel, kernel, border_mode='same'),
        BatchNormalization(),
        Activation('relu'),
        Convolution2D(64, kernel, kernel, border_mode='same'),
        BatchNormalization(),
        Activation('relu'),

        UpSampling2D(),
        Convolution2D(64, kernel, kernel, border_mode='same'),
        BatchNormalization(),
        Activation('relu'),
        Convolution2D(n_labels, 1, 1, border_mode='valid'),
        BatchNormalization(),
    ]
    autoencoder.decoding_layers = decoding_layers
    for l in autoencoder.decoding_layers:
        autoencoder.add(l)

    with open('model_5l.json', 'w') as outfile:
        outfile.write(json.dumps(json.loads(autoencoder.to_json()), indent=2))

    return autoencoder

In [ ]:
model=build_model()

In [ ]:
model.summary()

In [ ]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

import os
os.environ["PATH"] += os.pathsep + '''C:/Program Files (x86)/Graphviz2.38/bin/'''

!pip install graphviz
SVG(model_to_dot(model).create(prog='dot', format='svg'))

In [ ]:
from tensorflow.keras.optmizers import Adam

model.compile(optimizer=Adam(lr=3e-4), loss=dice_coef_loss,
                  metrics=[dice_coef, 'accuracy', precision, recall, f1score])

from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.15, random_state=2018)


model.fit(X_train, y_train, batch_size=train_batch_size, epochs=epochs, verbose=1, shuffle=True,
          callbacks=[csv_logger, model_checkpoint],validation_split=0.1)
